In [2]:
import sys
import os
from os import path

sys.path.append('C:\\Users\\Administrator\\Desktop\\风控产品\\risk_project')
from risk_models import *

In [6]:
sql = f"""\
SELECT ID,I_E_MARK,CUSTOMS_CODE,I_E_PORT,I_E_DATE,FIRST_DECL_TIME,TRAF_MODE,GOODS_OWNER FROM {TableList.FT_I_DTL_SEA_PRE_RECORDED.value} WHERE ISDELETED = 0 AND TRADE_CODE_SCC LIKE '91310115688774070B' AND I_E_DATE >= DATE'2019-02-01' AND I_E_DATE <= DATE'2021-02-01'
"""
CUS_SEA_HEAD_I = Read_Oracle().read_oracle(sql=sql, database='dbdw')

2021-06-04 13:48:25.784 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 17.63s


In [3]:
sql = f"""\
SELECT * FROM (select * from ODS_ZMXPQ.STOCK_BILL where org_code like '91310115688774070B' and iscurrent = 1 and dclcus_flag = 1) WHERE ROWNUM<=100000
"""
STOCK_BILL = Read_Oracle().read_oracle(sql=sql, database='dbods')

KeyboardInterrupt: 

In [28]:
STOCK_BILL['TRADE_TOTAL'] = STOCK_BILL['TRADE_TOTAL'].astype(float)
STOCK_BILL['QTY'] = STOCK_BILL['QTY'].astype(float)
STOCK_BILL['QTY_1'] = STOCK_BILL['QTY_1'].astype(float)
STOCK_BILL['QTY_2'] = STOCK_BILL['QTY_2'].astype(float)

stock_bill_sum = STOCK_BILL.groupby(['ORG_CODE','ENTRY_NO','ENTRY_GDS_SEQNO'])['QTY','QTY_1','QTY_2','TRADE_TOTAL'].sum()
stock_bill_sum = stock_bill_sum.reset_index()
stock_info = STOCK_BILL[['ORG_CODE','ENTRY_NO','ENTRY_GDS_SEQNO',
                                   'STOCK_BILL_TYPE','STOCK_TYPE','BILL_TYPE',
                                  'BUSINESS_TYPE','BILL_STATUS','CLASSIFY_TYPE',
                                  'SUPV_MODE','TRAF_MODE','I_E_PORT','CUSTOMS_CODE',
                                 'G_NAME','G_MODEL','ORIGIN_COUNTRY_CODE','HS_CODE','G_UNIT','UNIT_1',
                                  'UNIT_2','UNIT_PRICE','TRADE_CURR','STARTDT']].copy()

stock_info = stock_info.drop_duplicates(subset=['ORG_CODE','ENTRY_NO','ENTRY_GDS_SEQNO'])
stock_bill_clean = pd.merge(stock_bill_sum,stock_info,how='left',left_on=['ORG_CODE','ENTRY_NO','ENTRY_GDS_SEQNO'],
                right_on=['ORG_CODE','ENTRY_NO','ENTRY_GDS_SEQNO'])
stock_bill_clean['TRADE_CURR'].replace(to_replace=list(DIM_CURR['CURRENCY_CODE']),value=list(DIM_CURR['CUS_CURRENCY_CODE']),inplace=True)
stock_bill_clean['ENTRY_GDS_SEQNO'] = stock_bill_clean['ENTRY_GDS_SEQNO'].astype(int)


<ipython-input-28-662d05e3dc00>:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  stock_bill_sum = STOCK_BILL.groupby(['ORG_CODE','ENTRY_NO','ENTRY_GDS_SEQNO'])['QTY','QTY_1','QTY_2','TRADE_TOTAL'].sum()


In [29]:
STOCK_BILL.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 67 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   ORG_CODE             100000 non-null  object        
 1   ORG_KEY              100000 non-null  int64         
 2   ORG_NAME             100000 non-null  object        
 3   ID                   100000 non-null  object        
 4   STOCK_BILL_TYPE      100000 non-null  object        
 5   STOCK_BILL_NO        100000 non-null  object        
 6   STOCK_TYPE           100000 non-null  object        
 7   BILL_TYPE            100000 non-null  object        
 8   TRADE_CODE           52273 non-null   object        
 9   TRADE_NAME           52273 non-null   object        
 10  OWNER_NAME           0 non-null       object        
 11  BUSINESS_TYPE        100000 non-null  object        
 12  DECL_STOCK_DATE      55138 non-null   datetime64[ns]
 13  ACTRUAL_STOCK_D

In [32]:
STOCK_BILL['TRADE_TOTAL'] = STOCK_BILL['TRADE_TOTAL'].astype(float)
STOCK_BILL['QTY'] = STOCK_BILL['QTY'].astype(float)
STOCK_BILL['QTY_1'] = STOCK_BILL['QTY_1'].astype(float)
STOCK_BILL['QTY_2'] = STOCK_BILL['QTY_2'].astype(float)

stock_bill_sum = STOCK_BILL.groupby(['ORG_CODE','ENTRY_NO','ENTRY_GDS_SEQNO'])['QTY','QTY_1','QTY_2','TRADE_TOTAL'].sum()
stock_bill_sum = stock_bill_sum.reset_index()
stock_info = STOCK_BILL[['ORG_CODE','ENTRY_NO','ENTRY_GDS_SEQNO',
                                   'STOCK_BILL_TYPE','STOCK_TYPE','BILL_TYPE',
                                  'BUSINESS_TYPE','BILL_STATUS','CLASSIFY_TYPE',
                                  'SUPV_MODE','TRAF_MODE','I_E_PORT','CUSTOMS_CODE',
                                 'G_NAME','G_MODEL','ORIGIN_COUNTRY_CODE','HS_CODE','G_UNIT','UNIT_1',
                                  'UNIT_2','TRADE_CURR','STARTDT']].copy()

stock_info = stock_info.drop_duplicates(subset=['ORG_CODE','ENTRY_NO','ENTRY_GDS_SEQNO'])
stock_bill_clean = pd.merge(stock_bill_sum,stock_info,how='left',left_on=['ORG_CODE','ENTRY_NO','ENTRY_GDS_SEQNO'],
                right_on=['ORG_CODE','ENTRY_NO','ENTRY_GDS_SEQNO'])
stock_bill_clean['TRADE_CURR'].replace(to_replace=list(DIM_CURR['CURRENCY_CODE']),value=list(DIM_CURR['CUS_CURRENCY_CODE']),inplace=True)
stock_bill_clean['ENTRY_GDS_SEQNO'] = stock_bill_clean['ENTRY_GDS_SEQNO'].astype(int)

stock_bill_clean = stock_bill_clean.reset_index().rename(columns = {'index':'ID'})
now = datetime.datetime.strftime(datetime.datetime.now(), format='%Y-%m-%d %H:%M:%S')
stock_bill_clean['CHECK_TIME']=now
stock_bill_clean['CHECK_TIME']= pd.to_datetime(stock_bill_clean['CHECK_TIME'], format='%Y-%m-%d %H:%M:%S')
stock_bill_clean['TRADE_TOTAL'] = stock_bill_clean['TRADE_TOTAL'].astype(float)
stock_bill_clean['QTY'] = stock_bill_clean['QTY'].astype(float)
stock_bill_clean['QTY_1'] = stock_bill_clean['QTY_1'].astype(float)
stock_bill_clean['QTY_2'] = stock_bill_clean['QTY_2'].astype(float)

<ipython-input-32-11be24a62a63>:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  stock_bill_sum = STOCK_BILL.groupby(['ORG_CODE','ENTRY_NO','ENTRY_GDS_SEQNO'])['QTY','QTY_1','QTY_2','TRADE_TOTAL'].sum()


In [31]:
stock_bill_clean.columns

Index(['ORG_CODE', 'ENTRY_NO', 'ENTRY_GDS_SEQNO', 'QTY', 'QTY_1', 'QTY_2',
       'TRADE_TOTAL', 'STOCK_BILL_TYPE', 'STOCK_TYPE', 'BILL_TYPE',
       'BUSINESS_TYPE', 'BILL_STATUS', 'CLASSIFY_TYPE', 'SUPV_MODE',
       'TRAF_MODE', 'I_E_PORT', 'CUSTOMS_CODE', 'G_NAME', 'G_MODEL',
       'ORIGIN_COUNTRY_CODE', 'HS_CODE', 'G_UNIT', 'UNIT_1', 'UNIT_2',
       'UNIT_PRICE', 'TRADE_CURR', 'STARTDT'],
      dtype='object')

In [33]:
stock_bill_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25792 entries, 0 to 25791
Data columns (total 28 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID                   25792 non-null  int64         
 1   ORG_CODE             25792 non-null  object        
 2   ENTRY_NO             25792 non-null  object        
 3   ENTRY_GDS_SEQNO      25792 non-null  int32         
 4   QTY                  25792 non-null  float64       
 5   QTY_1                25792 non-null  float64       
 6   QTY_2                25792 non-null  float64       
 7   TRADE_TOTAL          25792 non-null  float64       
 8   STOCK_BILL_TYPE      25792 non-null  object        
 9   STOCK_TYPE           25792 non-null  object        
 10  BILL_TYPE            25792 non-null  object        
 11  BUSINESS_TYPE        25792 non-null  object        
 12  BILL_STATUS          0 non-null      object        
 13  CLASSIFY_TYPE        25792 non-

In [3]:
sql = f"""\
SELECT ID,I_E_MARK,CUSTOMS_CODE,I_E_PORT,I_E_DATE,FIRST_DECL_TIME,TRAF_MODE,GOODS_OWNER FROM DW_I_BASIC.FT_I_DTL_SEA_PRE_RECORDED WHERE ISDELETED = 0 AND TRADE_CODE_SCC LIKE '91310115688774070B'
"""
CUS_SEA_HEAD_I = Read_Oracle().read_oracle(sql=sql, database='dbdw')

sql = f"""\
SELECT HEAD_ID, ENTRY_NO, GOODS_NO, GOODS_CODE, GOODS_NAME, GOODS_MODEL, 
ORG_CODE_CY, G_QTY, G_UNIT, QTY_1, UNIT_1, QTY_2, UNIT_2, TRADE_CURR_CODE, 
DECL_TOTAL,'SEA' AS SRC FROM DW_I_BASIC.FT_I_DTL_SEA_LIST WHERE ISDELETED = 0
AND HEAD_ID IN
(SELECT ID FROM DW_I_BASIC.FT_I_DTL_SEA_PRE_RECORDED WHERE ISDELETED = 0 AND TRADE_CODE_SCC LIKE '91310115688774070B')
"""
CUS_SEA_LIST_I = Read_Oracle().read_oracle(sql=sql, database='dbdw')

sql = f"""\
SELECT ID,I_E_MARK,CUSTOMS_CODE,I_E_PORT,I_E_DATE,FIRST_DECL_TIME,TRAF_MODE,GOODS_OWNER FROM DW_I_BASIC.FT_I_DTL_OTR_PRE_RECORDED WHERE ISDELETED = 0 AND TRADE_CODE_SCC LIKE '91310115688774070B'
"""
CUS_OTR_HEAD_I = Read_Oracle().read_oracle(sql=sql, database='dbdw')

sql = f"""\
SELECT HEAD_ID, ENTRY_NO, GOODS_NO, GOODS_CODE, GOODS_NAME, GOODS_MODEL, 
ORG_CODE_CY, G_QTY, G_UNIT, QTY_1, UNIT_1, QTY_2, UNIT_2, TRADE_CURR_CODE, 
DECL_TOTAL,'OTR' AS SRC FROM DW_I_BASIC.FT_I_DTL_OTR_LIST WHERE ISDELETED = 0
AND HEAD_ID IN
(SELECT ID FROM DW_I_BASIC.FT_I_DTL_OTR_PRE_RECORDED WHERE ISDELETED = 0 AND TRADE_CODE_SCC LIKE '91310115688774070B')
"""
CUS_OTR_LIST_I = Read_Oracle().read_oracle(sql=sql, database='dbdw')

2021-06-04 09:26:30.338 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 38.08s
2021-06-04 09:27:32.672 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 62.31s
2021-06-04 09:30:11.865 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 159.1s
2021-06-04 09:33:05.317 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 173.4s


In [4]:
CUS_SEA_IMPORT = pd.merge(CUS_SEA_HEAD_I,CUS_SEA_LIST_I,how='inner',left_on=['ID'],right_on=['HEAD_ID'])
CUS_SEA_IMPORT = CUS_SEA_IMPORT.drop_duplicates(subset=['ENTRY_NO','GOODS_NO'])

CUS_OTR_IMPORT = pd.merge(CUS_OTR_HEAD_I,CUS_OTR_LIST_I,how='inner',left_on=['ID'],right_on=['HEAD_ID'])
CUS_OTR_IMPORT = CUS_OTR_IMPORT.drop_duplicates(subset=['ENTRY_NO','GOODS_NO'])

In [5]:
CUS_DATA = pd.concat([CUS_SEA_IMPORT,CUS_OTR_IMPORT])

In [7]:
CUS_DATA[pd.isnull(CUS_DATA['SRC'])]

,ID,I_E_MARK,CUSTOMS_CODE,I_E_PORT,I_E_DATE,FIRST_DECL_TIME,TRAF_MODE,GOODS_OWNER,HEAD_ID,ENTRY_NO,...,ORG_CODE_CY,G_QTY,G_UNIT,QTY_1,UNIT_1,QTY_2,UNIT_2,TRADE_CURR_CODE,DECL_TOTAL,SRC


In [21]:
CUS_DATA.columns

Index(['ID', 'I_E_MARK', 'CUSTOMS_CODE', 'I_E_PORT', 'I_E_DATE',
       'FIRST_DECL_TIME', 'TRAF_MODE', 'GOODS_OWNER', 'HEAD_ID', 'ENTRY_NO',
       'GOODS_NO', 'GOODS_CODE', 'GOODS_NAME', 'GOODS_MODEL', 'ORG_CODE_CY',
       'G_QTY', 'G_UNIT', 'QTY_1', 'UNIT_1', 'QTY_2', 'UNIT_2',
       'TRADE_CURR_CODE', 'DECL_TOTAL', 'SRC'],
      dtype='object')

In [20]:
CUS_DATA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23570 entries, 0 to 23569
Data columns (total 24 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   ID               23570 non-null  int64         
 1   I_E_MARK         23570 non-null  object        
 2   CUSTOMS_CODE     23570 non-null  object        
 3   I_E_PORT         23570 non-null  object        
 4   I_E_DATE         23570 non-null  datetime64[ns]
 5   FIRST_DECL_TIME  23570 non-null  datetime64[ns]
 6   TRAF_MODE        23570 non-null  object        
 7   GOODS_OWNER      23570 non-null  object        
 8   HEAD_ID          23570 non-null  int64         
 9   ENTRY_NO         23570 non-null  object        
 10  GOODS_NO         23570 non-null  int64         
 11  GOODS_CODE       23570 non-null  object        
 12  GOODS_NAME       23570 non-null  object        
 13  GOODS_MODEL      23570 non-null  object        
 14  ORG_CODE_CY      23570 non-null  objec

0        41169566
1        41169566
2        41169566
3        41183823
4        41183823
          ...    
8340    103269676
8341    103280274
8342    103280274
8343    103280274
8344    103380778
Name: ID, Length: 23570, dtype: int64

In [7]:
CUS_OTR_IMPORT.shape

(8345, 24)

In [5]:
CUS_DATA.shape

(23570, 24)

In [2]:
sql = f"""\
SELECT * FROM (select * from ODS_ZMXPQ.STOCK_BILL where org_code like '91310115688774070B' and iscurrent = 1 and dclcus_flag = 1) WHERE ROWNUM<=100000
"""
STOCK_BILL = Read_Oracle().read_oracle(sql=sql, database='dbods')

2021-06-03 09:11:03.711 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 11.05s


In [79]:
sql = f"""\
SELECT ID,I_E_MARK,CUSTOMS_CODE,I_E_PORT,I_E_DATE,FIRST_DECL_TIME,TRAF_MODE,GOODS_OWNER,DEPARTURE_CY FROM DW_I_BASIC.FT_I_DTL_SEA_PRE_RECORDED WHERE ISDELETED = 0 AND TRADE_CODE_SCC LIKE '91310115688774070B'
"""
CUS_SEA_HEAD_I = Read_Oracle().read_oracle(sql=sql, database='dbdw')

2021-06-03 10:26:02.673 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 21.19s


In [80]:
sql = f"""\
SELECT HEAD_ID, ENTRY_NO, GOODS_NO, GOODS_CODE, GOODS_NAME, GOODS_MODEL, 
ORG_CODE_CY, G_QTY, G_UNIT, QTY_1, UNIT_1, QTY_2, UNIT_2, TRADE_CURR_KEY, TRADE_CURR_CODE, 
DECL_PRICE, DECL_TOTAL, GOODS_GROSS_VALUE_RMB, DEST_CY, GOODS_GROSS_VALUE_USD FROM DW_I_BASIC.FT_I_DTL_SEA_LIST WHERE ISDELETED = 0
AND HEAD_ID IN
(SELECT ID FROM DW_I_BASIC.FT_I_DTL_SEA_PRE_RECORDED WHERE ISDELETED = 0 AND TRADE_CODE_SCC LIKE '91310115688774070B')
"""
CUS_SEA_LIST_I = Read_Oracle().read_oracle(sql=sql, database='dbdw')

2021-06-03 10:26:22.024 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 19.33s


In [11]:
sql = f"""\
SELECT CY_CODE_ALPHY3,CY_CODE_CUS_NUMBER FROM DIM.DIM_COUNTRY WHERE ISCURRENT = 1 AND REGEXP_LIKE(CY_CODE_CUS_NUMBER,'^[0-9]*$') AND CY_CODE_CUS_NUMBER NOT LIKE '299' 
"""
DIM_COUNTRY = Read_Oracle().read_oracle(sql=sql, database='dbdw')

2021-06-04 09:44:57.677 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.497s


In [9]:
sql = f"""\
SELECT CUS_CURRENCY_CODE,CURRENCY_CODE FROM DIM.MAP_GJ_CURRENCY
"""
DIM_CURR = Read_Oracle().read_oracle(sql=sql, database='dbdw')

2021-06-04 09:40:45.162 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.126s


In [82]:
CUS_SEA_IMPORT = pd.merge(CUS_SEA_HEAD_I,CUS_SEA_LIST_I,how='inner',left_on=['ID'],right_on=['HEAD_ID'])

In [90]:
CUS_SEA_IMPORT = CUS_SEA_IMPORT.drop_duplicates(subset=['ENTRY_NO','GOODS_NO'])

In [100]:
entry_list = CUS_SEA_IMPORT[['ENTRY_NO','GOODS_NO']].copy()

In [110]:
entry_list = entry_list.drop_duplicates()
entry_list['merge_flag'] = 1

In [106]:
test2['ENTRY_GDS_SEQNO'] = test2['ENTRY_GDS_SEQNO'].astype(int)

In [111]:
test_merge = pd.merge(test2,entry_list,how='left',left_on=['ENTRY_NO','ENTRY_GDS_SEQNO'],right_on=['ENTRY_NO','GOODS_NO'])

In [114]:
test_merge.shape

(25792, 36)

In [115]:
test_merge = test_merge[-pd.isnull(test_merge['merge_flag'])].copy()

In [174]:
A = test_merge[['ENTRY_NO','ENTRY_GDS_SEQNO','TRAF_MODE']].copy()
A.columns = ['ENTRY_NO','GOODS_NO','test_1']

In [175]:
verify_data = pd.merge(A,CUS_SEA_IMPORT,how='inner',on=['ENTRY_NO','GOODS_NO'])

In [139]:
verify_data.shape

(1055, 32)

In [176]:
verify_data = verify_data[verify_data['test_1'] != verify_data['TRAF_MODE']].copy()

In [177]:
verify_data[['test_1','GOODS_MODEL']]

,test_1,GOODS_MODEL


In [170]:
verify_data.loc[0,'test_1']

'4|3|用于港口机械，堆高机、集装箱正面吊、起重机等，能量转换提供动力|235KW|VOLVO PENTA|TAD1150VE|2100RPM||'

In [171]:
verify_data.loc[0,'GOODS_MODEL']

'4|3|用于港口机械,堆高机、集装箱正面吊、起重机等,能量转换提供动力|235KW|VOLVO PENTA|TAD1150VE|2100RPM'

In [140]:
verify_data = verify_data[verify_data['G_UNIT'] != verify_data['test_1']].copy()

In [141]:
verify_data.shape

(37, 32)

In [144]:
verify_data = verify_data[verify_data['G_UNIT'] != verify_data['test_2']].copy()

In [145]:
verify_data.shape

(37, 32)

In [146]:
verify_data = verify_data[verify_data['G_UNIT'] != verify_data['test_3']].copy()

In [ ]:
verify_data.shape

In [147]:
verify_data.shape

(37, 32)

In [149]:
verify_data = verify_data[verify_data['UNIT_1'] != verify_data['test_1']].copy()
verify_data = verify_data[verify_data['UNIT_1'] != verify_data['test_2']].copy()
verify_data = verify_data[verify_data['UNIT_1'] != verify_data['test_3']].copy()

In [150]:
verify_data.shape

(4, 32)

In [151]:
verify_data = verify_data[verify_data['UNIT_2'] != verify_data['test_1']].copy()
verify_data = verify_data[verify_data['UNIT_2'] != verify_data['test_2']].copy()
verify_data = verify_data[verify_data['UNIT_2'] != verify_data['test_3']].copy()

In [152]:
verify_data.shape

(0, 32)

In [194]:
A = test_merge[['ENTRY_NO','ENTRY_GDS_SEQNO','ORIGIN_COUNTRY_CODE']].copy()
A.columns = ['ENTRY_NO','GOODS_NO','test_1']
verify_data = pd.merge(A,CUS_SEA_IMPORT,how='inner',on=['ENTRY_NO','GOODS_NO'])
verify_data = verify_data[verify_data['test_1'] != verify_data['ORG_CODE_CY']].copy()

In [195]:
verify_data[['test_1','ORG_CODE_CY']]

,test_1,ORG_CODE_CY
52,142,328
53,328,330
54,328,330
62,330,142
271,330,327
...,...,...
983,142,330
984,336,330
985,330,303
990,344,502


In [99]:
test2.columns

Index(['ORG_CODE', 'ENTRY_NO', 'ENTRY_GDS_SEQNO', 'QTY', 'QTY_1', 'QTY_2',
       'NET_WT', 'GROSS_WT', 'TRADE_TOTAL', 'STOCK_BILL_TYPE', 'STOCK_TYPE',
       'BILL_TYPE', 'OWNER_NAME', 'BUSINESS_TYPE', 'BILL_STATUS',
       'CLASSIFY_TYPE', 'SUPV_MODE', 'TRAF_MODE', 'I_E_PORT', 'CUSTOMS_CODE',
       'DESTINATION_CODE', 'STSHIP_TRSARV_CODE', 'TRADE_COUNTRY_CODE',
       'TRANS_MODE', 'G_NAME', 'G_MODEL', 'ORIGIN_COUNTRY_CODE', 'HS_CODE',
       'G_UNIT', 'UNIT_1', 'UNIT_2', 'UNIT_PRICE', 'TRADE_CURR', 'STARTDT'],
      dtype='object')

In [98]:
CUS_SEA_IMPORT.columns

Index(['ID', 'I_E_MARK', 'CUSTOMS_CODE', 'I_E_PORT', 'I_E_DATE',
       'FIRST_DECL_TIME', 'TRAF_MODE', 'GOODS_OWNER', 'DEPARTURE_CY',
       'HEAD_ID', 'ENTRY_NO', 'GOODS_NO', 'GOODS_CODE', 'GOODS_NAME',
       'GOODS_MODEL', 'ORG_CODE_CY', 'G_QTY', 'G_UNIT', 'QTY_1', 'UNIT_1',
       'QTY_2', 'UNIT_2', 'TRADE_CURR_KEY', 'TRADE_CURR_CODE', 'DECL_PRICE',
       'DECL_TOTAL', 'GOODS_GROSS_VALUE_RMB', 'DEST_CY',
       'GOODS_GROSS_VALUE_USD'],
      dtype='object')

In [97]:
CUS_SEA_IMPORT['DEPARTURE_CY'].replace(to_replace=list(DIM_COUNTRY['CY_CODE_ALPHY3']),value=list(DIM_COUNTRY['CY_CODE_CUS_NUMBER']),inplace=True)

CUS_SEA_IMPORT['ORG_CODE_CY'].replace(to_replace=list(DIM_COUNTRY['CY_CODE_ALPHY3']),value=list(DIM_COUNTRY['CY_CODE_CUS_NUMBER']),inplace=True)

CUS_SEA_IMPORT['DEST_CY'].replace(to_replace=list(DIM_COUNTRY['CY_CODE_ALPHY3']),value=list(DIM_COUNTRY['CY_CODE_CUS_NUMBER']),inplace=True)

CUS_SEA_IMPORT['TRADE_CURR_CODE'].replace(to_replace=list(DIM_CURR['CURRENCY_CODE']),value=list(DIM_CURR['CUS_CURRENCY_CODE']),inplace=True)

In [10]:
CUS_DATA['TRADE_CURR_CODE'].replace(to_replace=list(DIM_CURR['CURRENCY_CODE']),value=list(DIM_CURR['CUS_CURRENCY_CODE']),inplace=True)

In [12]:
CUS_DATA['ORG_CODE_CY'].replace(to_replace=list(DIM_COUNTRY['CY_CODE_ALPHY3']),value=list(DIM_COUNTRY['CY_CODE_CUS_NUMBER']),inplace=True)

In [14]:
CUS_DATA = CUS_DATA.drop(columns=['ID'])

In [16]:
CUS_DATA = CUS_DATA.reset_index().rename(columns = {'index':'ID'})

In [19]:
CUS_DATA['ID'].unique()

array([    0,     1,     2, ..., 15222, 15223, 15224], dtype=int64)

In [89]:
CUS_SEA_IMPORT.shape

(15224, 29)

In [91]:
CUS_SEA_IMPORT.shape

(15224, 29)

In [59]:
del CUS_SEA_HEAD_I,CUS_SEA_LIST_I

NameError: name 'CUS_SEA_HEAD_I' is not defined

In [60]:
import gc
gc.collect()

116

In [13]:
import os
os.getpid()

7264

In [3]:
stock_bill_sum = STOCK_BILL.groupby(['ORG_CODE','ENTRY_NO','ENTRY_GDS_SEQNO'])['QTY','QTY_1','QTY_2','NET_WT','GROSS_WT','TRADE_TOTAL'].sum()

stock_bill_sum = stock_bill_sum.reset_index()

stock_info = STOCK_BILL[['ORG_CODE','ENTRY_NO','ENTRY_GDS_SEQNO',
                                   'STOCK_BILL_TYPE','STOCK_TYPE','BILL_TYPE',
                                  'OWNER_NAME','BUSINESS_TYPE','BILL_STATUS','CLASSIFY_TYPE',
                                  'SUPV_MODE','TRAF_MODE','I_E_PORT','CUSTOMS_CODE','DESTINATION_CODE',
                                  'STSHIP_TRSARV_CODE','TRADE_COUNTRY_CODE','TRANS_MODE','G_NAME',
                                  'G_MODEL','ORIGIN_COUNTRY_CODE','HS_CODE','G_UNIT','UNIT_1',
                                  'UNIT_2','UNIT_PRICE','TRADE_CURR','STARTDT']].copy()

stock_info = stock_info.drop_duplicates(subset=['ORG_CODE','ENTRY_NO','ENTRY_GDS_SEQNO'])

stock_bill_clean = pd.merge(stock_bill_sum,stock_info,how='left',left_on=['ORG_CODE','ENTRY_NO','ENTRY_GDS_SEQNO'],
                right_on=['ORG_CODE','ENTRY_NO','ENTRY_GDS_SEQNO'])

stock_bill_clean['TRADE_COUNTRY_CODE'].replace(to_replace=list(DIM_COUNTRY['CY_CODE_ALPHY3']),value=list(DIM_COUNTRY['CY_CODE_CUS_NUMBER']),inplace=True)

stock_bill_clean['ORIGIN_COUNTRY_CODE'].replace(to_replace=list(DIM_COUNTRY['CY_CODE_ALPHY3']),value=list(DIM_COUNTRY['CY_CODE_CUS_NUMBER']),inplace=True)

stock_bill_clean['DESTINATION_CODE'].replace(to_replace=list(DIM_COUNTRY['CY_CODE_ALPHY3']),value=list(DIM_COUNTRY['CY_CODE_CUS_NUMBER']),inplace=True)

stock_bill_clean['STSHIP_TRSARV_CODE'].replace(to_replace=list(DIM_COUNTRY['CY_CODE_ALPHY3']),value=list(DIM_COUNTRY['CY_CODE_CUS_NUMBER']),inplace=True)

stock_bill_clean['TRADE_CURR'].replace(to_replace=list(DIM_CURR['CURRENCY_CODE']),value=list(DIM_CURR['CUS_CURRENCY_CODE']),inplace=True)

stock_bill_clean['ENTRY_GDS_SEQNO'] = stock_bill_clean['ENTRY_GDS_SEQNO'].astype(int)

<ipython-input-3-2abd7449470a>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  test1 = STOCK_BILL.groupby(['ORG_CODE','ENTRY_NO','ENTRY_GDS_SEQNO'])['QTY','QTY_1','QTY_2','NET_WT','GROSS_WT','TRADE_TOTAL'].sum()


In [198]:
STOCK_BILL['TRADE_TOTAL'] = STOCK_BILL['TRADE_TOTAL'].astype(float)

In [72]:
stock_bill_clean.columns

Index(['ORG_CODE', 'ENTRY_NO', 'ENTRY_GDS_SEQNO', 'QTY', 'QTY_1', 'QTY_2',
       'NET_WT', 'GROSS_WT', 'TRADE_TOTAL', 'STOCK_BILL_TYPE', 'STOCK_TYPE',
       'BILL_TYPE', 'OWNER_NAME', 'BUSINESS_TYPE', 'BILL_STATUS',
       'CLASSIFY_TYPE', 'SUPV_MODE', 'TRAF_MODE', 'I_E_PORT', 'CUSTOMS_CODE',
       'DESTINATION_CODE', 'STSHIP_TRSARV_CODE', 'TRADE_COUNTRY_CODE',
       'TRANS_MODE', 'G_NAME', 'G_MODEL', 'ORIGIN_COUNTRY_CODE', 'HS_CODE',
       'G_UNIT', 'UNIT_1', 'UNIT_2', 'UNIT_PRICE', 'TRADE_CURR', 'STARTDT'],
      dtype='object')

In [95]:
test2['STSHIP_TRSARV_CODE']

0         142
1        None
2        None
3        None
4        None
         ... 
25787    None
25788     601
25789     330
25790     309
25791     309
Name: STSHIP_TRSARV_CODE, Length: 25792, dtype: object

In [94]:
test2['STSHIP_TRSARV_CODE'].replace(to_replace=list(DIM_COUNTRY['CY_CODE_ALPHY3']),value=list(DIM_COUNTRY['CY_CODE_CUS_NUMBER']),inplace=True)


In [47]:
import re

In [51]:
def is_num(x):
    if pd.isnull(x):
        return False
    else:
        if re.search('^[0-9]*$',x):
            return True
        else:
            return False

def is_num_row(row):
    return row.apply(is_num)

In [52]:
is_num_row(test2['TRADE_COUNTRY_CODE'])

0        False
1         True
2         True
3         True
4         True
         ...  
25787     True
25788     True
25789     True
25790     True
25791     True
Name: TRADE_COUNTRY_CODE, Length: 25792, dtype: bool

In [41]:
CUS_SEA_IMPORT

,ID,I_E_MARK,CUSTOMS_CODE,I_E_PORT,I_E_DATE,FIRST_DECL_TIME,TRAF_MODE,GOODS_OWNER,DEPARTURE_CY,HEAD_ID,...,NO_DGRGD_FLAG,CIQ_DOM_DEST_CODE,CIQ_GOODS_SPEC,CIQ_GOODS_MODEL,CIQ_GOODS_BRAND,PRODUCE_DATE,PRODUCE_BATCH_NO,CIQ_NAME,PRODUCER_REGISTER_NO,PRODUCER_REGISTER_NAME
0,41169566,I,2249,2248,2018-08-12,2018-08-13 11:23:55,2,上海综合保税区国际物流有限公司,NZL,41169566,...,None,None,None,None,None,None,None,None,None,None
1,41169566,I,2249,2248,2018-08-12,2018-08-13 11:23:55,2,上海综合保税区国际物流有限公司,NZL,41169566,...,None,None,None,None,None,None,None,None,None,None
2,41169566,I,2249,2248,2018-08-12,2018-08-13 11:23:55,2,上海综合保税区国际物流有限公司,NZL,41169566,...,None,None,None,None,None,None,None,None,None,None
3,41183823,I,2249,2248,2018-08-12,2018-08-13 13:52:47,2,上海综合保税区国际物流有限公司,NZL,41183823,...,None,None,None,None,None,None,None,None,None,None
4,41183823,I,2249,2248,2018-08-12,2018-08-13 13:52:47,2,上海综合保税区国际物流有限公司,NZL,41183823,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15219,103004540,I,2249,2225,2021-05-28,2021-05-28 14:33:17,2,上海综合保税区国际物流有限公司,KOR,103004540,...,None,310115,None,None,None,None,None,光缆,None,None
15220,103005058,I,2249,2248,2021-05-27,2021-05-28 14:36:30,2,上海综合保税区国际物流有限公司,SVN,103005058,...,None,310115,None,None,None,None,None,800毫升＜排量装往复式活塞内燃发动机摩托车及脚踏两用车,None,None
15221,102732184,I,2249,2225,2021-05-24,2021-05-25 13:26:57,2,上海综合保税区国际物流有限公司,SGP,102732184,...,None,310115,None,None,None,None,None,其他未列名硫化橡胶制品(硬质橡胶除外)(硫化橡胶制品),None,None
15222,102732184,I,2249,2225,2021-05-24,2021-05-25 13:26:57,2,上海综合保税区国际物流有限公司,SGP,102732184,...,None,310115,None,None,None,None,None,其他工业用钢铁制品,None,None


In [14]:
STOCK_BILL.shape

(100000, 67)

In [58]:
del STOCK_BILL,CUS_SEA_IMPORT

NameError: name 'STOCK_BILL' is not defined

In [35]:
CUS_SEA_IMPORT.columns

Index(['ID_x', 'I_E_MARK', 'CUSTOMS_CODE', 'I_E_PORT', 'I_E_DATE',
       'FIRST_DECL_TIME', 'TRAF_MODE', 'GOODS_OWNER', 'DEPARTURE_CY', 'ID_y',
       'HEAD_ID', 'VSL_VOY_KEY', 'VSL_NAME_KEY', 'ACCURATE_VSLNAME',
       'ACCURATE_VOYAGE', 'VSL_NAME', 'VOYAGE', 'BL_NO_KEY', 'BL_NO',
       'ENTRY_NO', 'GOODS_NO', 'GOODS_CODE_KEY', 'GOODS_CODE', 'GOODS_NAME',
       'GOODS_MODEL', 'ORI_AREA_CODE_KEY', 'ORG_CODE_CY', 'CONTR_ITEM',
       'G_QTY', 'G_UNIT', 'QTY_1', 'UNIT_1', 'QTY_2', 'UNIT_2',
       'TRADE_CURR_KEY', 'TRADE_CURR_CODE', 'DECL_PRICE', 'DECL_TOTAL',
       'EXCHANGE_RATE', 'GOODS_GROSS_VALUE_RMB', 'USER_TO', 'DUTY_MODE_KEY',
       'DUTY_MODE', 'WORK_USD', 'PDRT_NO', 'PROCESS_MATERIAL_PRDT_NO',
       'DEST_CY_KEY', 'DEST_CY', 'ISDELETED', 'SYSTEMKEY', 'FEED', 'FEEDKEY',
       'FEEDVALUE', 'LASTUPDATEDDT', 'FIRSTINSERTEDDT', 'ID_FACTTECHNICALID',
       'EXCHANGE_RATE_USD', 'GOODS_GROSS_VALUE_USD', 'G_UNIT_KEY',
       'UNIT_1_KEY', 'UNIT_2_KEY', 'CUS_DOM_DEST_CODE_KEY',


In [44]:
STOCK_BILL['ENTRY_GDS_SEQNO'].unique()

array(['11', '1', '20', '21', '7', '2', '8', '6', '3', '5', '4', '22',
       '13', '17', '10', '9', '12', '0', '14', '19', '18', '16', '15',
       '26', '27', '28', '29', '30', '31', '32', '35', '36', '23', '33',
       '24', '25', '39', '38', '34', '40', '41', '37', '42', None, '46',
       '47', '43', '48', '49', '44', '45', '50'], dtype=object)

In [46]:
STOCK_BILL['ENTRY_GDS_SEQNO'] = STOCK_BILL['ENTRY_GDS_SEQNO'].astype(int)

In [47]:
test = pd.merge(STOCK_BILL,CUS_SEA_IMPORT,how='left',left_on=['ENTRY_NO','BILL_NO','ENTRY_GDS_SEQNO'],right_on=['ENTRY_NO','BL_NO','GOODS_NO'])

In [56]:
test = test[-pd.isnull(test['FIRST_DECL_TIME'])].copy()

In [7]:
STOCK_BILL.columns

Index(['ORG_CODE', 'ORG_KEY', 'ORG_NAME', 'ID', 'STOCK_BILL_TYPE',
       'STOCK_BILL_NO', 'STOCK_TYPE', 'BILL_TYPE', 'TRADE_CODE', 'TRADE_NAME',
       'OWNER_NAME', 'BUSINESS_TYPE', 'DECL_STOCK_DATE', 'ACTRUAL_STOCK_DATE',
       'BILL_STATUS', 'CLASSIFY_TYPE', 'SUPV_MODE', 'TRAF_MODE', 'I_E_PORT',
       'CUSTOMS_CODE', 'DESTINATION_CODE', 'STSHIP_TRSARV_CODE',
       'TRADE_COUNTRY_CODE', 'DISTRICT_CODE', 'TRANS_MODE', 'COP_G_NO',
       'G_NAME', 'G_MODEL', 'ORIGIN_COUNTRY_CODE', 'HS_CODE', 'CIQ_CODE',
       'QTY', 'G_UNIT', 'QTY_1', 'UNIT_1', 'QTY_2', 'UNIT_2', 'NET_WT',
       'GROSS_WT', 'WH_NO', 'WH_LOC', 'ORDER_NO', 'TRADE_CURR', 'UNIT_PRICE',
       'RLT_INVT_NO', 'TRANSIT_TIME', 'DCLCUS_FLAG', 'ENTRY_NO', 'BILL_NO',
       'PASS_TIME', 'RLT_ENTRY_NO', 'ENTRY_GDS_SEQNO', 'INPUTDBTIME',
       'SNAPSHOTID', 'ISCURRENT', 'ISDELETED', 'SYSTEMKEY', 'FEED', 'FEEDKEY',
       'FEEDVALUE', 'LASTUPDATEDDT', 'STARTDT', 'ENDDT', 'CAPXACTION',
       'BILL_DETAIL_SEQNO', 'TRADE_TOTAL'

In [21]:
STOCK_BILL['ENTRY_GDS_SEQNO']

0    6
1    5
2    8
3    6
Name: ENTRY_GDS_SEQNO, dtype: object

,ORG_CODE,ORG_KEY,ORG_NAME,ID,STOCK_BILL_TYPE,STOCK_BILL_NO,STOCK_TYPE,BILL_TYPE,TRADE_CODE,TRADE_NAME,...,FEED,FEEDKEY,FEEDVALUE,LASTUPDATEDDT,STARTDT,ENDDT,CAPXACTION,BILL_DETAIL_SEQNO,TRADE_TOTAL,ID_FACTTECHNICALID
0,91310115688774070B,3,上海综合保税区国际物流有限公司,1358372,1,LGR-2020050305,1,1,3116610013,上海综合保税区国际物流有限公司,...,STOCK_BILL_NEW,ID,262407,2020-08-06 16:02:21.915915,2020-05-20 17:02:36.959563,None,U,I571367,1921.2000,24C1C04113FC22CBA6C5A407CD65F6E7
1,91310115688774070B,3,上海综合保税区国际物流有限公司,1358375,1,LGR-2020050305,1,1,3116610013,上海综合保税区国际物流有限公司,...,STOCK_BILL_NEW,ID,262410,2020-08-06 16:02:21.915915,2020-05-20 17:02:36.959563,None,U,I571370,10507.0000,3A209190D1FBA8E80045784517933622
2,91310115688774070B,3,上海综合保税区国际物流有限公司,1358363,1,LGR-2020050305,1,1,3116610013,上海综合保税区国际物流有限公司,...,STOCK_BILL_NEW,ID,262415,2020-08-06 16:02:21.915915,2020-05-20 17:02:36.959563,None,U,I571375,64.6200,F11D1E4B54E5E4725D71D99572A03780
3,91310115688774070B,3,上海综合保税区国际物流有限公司,1358371,1,LGR-2020050305,1,1,3116610013,上海综合保税区国际物流有限公司,...,STOCK_BILL_NEW,ID,262421,2020-08-06 16:02:21.915915,2020-05-20 17:02:36.959563,None,U,I571381,3365.5000,C7E3934312BB7DA7AA95C2D70C18C24B


In [64]:
test.columns

Index(['ORG_CODE', 'ORG_KEY', 'ORG_NAME', 'ID_x', 'STOCK_BILL_TYPE',
       'STOCK_BILL_NO', 'STOCK_TYPE', 'BILL_TYPE', 'TRADE_CODE', 'TRADE_NAME',
       ...
       'NO_DGRGD_FLAG', 'CIQ_DOM_DEST_CODE', 'CIQ_GOODS_SPEC',
       'CIQ_GOODS_MODEL', 'CIQ_GOODS_BRAND', 'PRODUCE_DATE',
       'PRODUCE_BATCH_NO', 'CIQ_NAME', 'PRODUCER_REGISTER_NO',
       'PRODUCER_REGISTER_NAME'],
      dtype='object', length=154)

In [62]:
df = test.groupby(['ORG_CODE','BILL_NO','ENTRY_NO','ENTRY_GDS_SEQNO'])

In [6]:
import sys
import os
from os import path

sys.path.append('C:\\Users\\Administrator\\Desktop\\风控产品\\risk_project')
# sys.path.append(path.dirname(path.dirname(path.dirname(os.getcwd()))))
from risk_models import *


class WarehouseCleanWh1(object):
    def __init__(self, org_code, params, base_time, child_task_id):
        self.model_code = 'WAREHOUSE'
        self.child_model_code = 'WH1'
        self.child_task_id = child_task_id
        Risk_logger(child_task_id=self.child_task_id, exec_status=None)

        # 参数读取
        self.org_code = org_code
        self.startdt = datetime.datetime.strptime(base_time,"%Y-%m-%d %H:%M:%S") + datetime.timedelta(days=-330)
        self.startdt2 = datetime.datetime.strptime(base_time,"%Y-%m-%d %H:%M:%S") + datetime.timedelta(days=-360)
        self.startdt = self.startdt.strftime('%Y-%m-%d')
        self.startdt2 = self.startdt2.strftime('%Y-%m-%d')
        self.enddt = datetime.datetime.strptime(base_time,"%Y-%m-%d %H:%M:%S").strftime('%Y-%m-%d')       


    def clean_wh1(self):
        # 为了测试有限制行数，记得去掉行数限制
        sql = f"""\
        SELECT * FROM (select * from {TableList.WAREHOUSE_STOCK_BILL.value} where org_code like '{self.org_code}' and iscurrent = 1 and dclcus_flag = 1) WHERE ROWNUM<=100000
        """
        STOCK_BILL = Read_Oracle().read_oracle(sql=sql, database='dbods')

        # 报关单语句应加上时间限制，用 startdt2
        sql = f"""\
        SELECT ID,I_E_MARK,CUSTOMS_CODE,I_E_PORT,I_E_DATE,FIRST_DECL_TIME,TRAF_MODE,GOODS_OWNER FROM {TableList.FT_I_DTL_SEA_PRE_RECORDED.value} WHERE ISDELETED = 0 AND TRADE_CODE_SCC LIKE '{self.org_code}' AND I_E_DATE >= DATE'{self.startdt2}' AND I_E_DATE <= DATE'{self.enddt}'
        """
        CUS_SEA_HEAD_I = Read_Oracle().read_oracle(sql=sql, database='dbdw')

        sql = f"""\
        SELECT HEAD_ID, ENTRY_NO, GOODS_NO, GOODS_CODE, GOODS_NAME, GOODS_MODEL, 
        ORG_CODE_CY, G_QTY, G_UNIT, QTY_1, UNIT_1, QTY_2, UNIT_2, TRADE_CURR_CODE, 
        DECL_TOTAL,'SEA' AS SRC FROM {TableList.FT_I_DTL_SEA_LIST.value} WHERE ISDELETED = 0
        AND HEAD_ID IN
        (SELECT ID FROM {TableList.FT_I_DTL_SEA_PRE_RECORDED.value} WHERE ISDELETED = 0 AND TRADE_CODE_SCC LIKE '{self.org_code}' AND I_E_DATE >= DATE'{self.startdt2}' AND I_E_DATE <= DATE'{self.enddt}')
        """
        CUS_SEA_LIST_I = Read_Oracle().read_oracle(sql=sql, database='dbdw')

        sql = f"""\
        SELECT ID,I_E_MARK,CUSTOMS_CODE,I_E_PORT,I_E_DATE,FIRST_DECL_TIME,TRAF_MODE,GOODS_OWNER FROM {TableList.FT_I_DTL_OTR_PRE_RECORDED.value} WHERE ISDELETED = 0 AND TRADE_CODE_SCC LIKE '{self.org_code}' AND I_E_DATE >= DATE'{self.startdt2}' AND I_E_DATE <= DATE'{self.enddt}'
        """
        CUS_OTR_HEAD_I = Read_Oracle().read_oracle(sql=sql, database='dbdw')

        sql = f"""\
        SELECT HEAD_ID, ENTRY_NO, GOODS_NO, GOODS_CODE, GOODS_NAME, GOODS_MODEL, 
        ORG_CODE_CY, G_QTY, G_UNIT, QTY_1, UNIT_1, QTY_2, UNIT_2, TRADE_CURR_CODE, 
        DECL_TOTAL,'OTR' AS SRC FROM {TableList.FT_I_DTL_OTR_LIST.value} WHERE ISDELETED = 0
        AND HEAD_ID IN
        (SELECT ID FROM {TableList.FT_I_DTL_OTR_PRE_RECORDED.value} WHERE ISDELETED = 0 AND TRADE_CODE_SCC LIKE '{self.org_code}' AND I_E_DATE >= DATE'{self.startdt2}' AND I_E_DATE <= DATE'{self.enddt}')
        """
        CUS_OTR_LIST_I = Read_Oracle().read_oracle(sql=sql, database='dbdw')

        sql = f"""\
        SELECT ID,I_E_MARK,CUSTOMS_CODE,I_E_PORT,I_E_DATE,FIRST_DECL_TIME,TRAF_MODE,GOODS_OWNER FROM {TableList.FT_E_DTL_SEA_PRE_RECORDED.value} WHERE ISDELETED = 0 AND TRADE_CODE_SCC LIKE '{self.org_code}' AND I_E_DATE >= DATE'{self.startdt2}' AND I_E_DATE <= DATE'{self.enddt}'
        """
        CUS_SEA_HEAD_E = Read_Oracle().read_oracle(sql=sql, database='dbdw')

        sql = f"""\
        SELECT HEAD_ID, ENTRY_NO, GOODS_NO, GOODS_CODE, GOODS_NAME, GOODS_MODEL, 
        ORG_CODE_CY, G_QTY, G_UNIT, QTY_1, UNIT_1, QTY_2, UNIT_2, TRADE_CURR_CODE, 
        DECL_TOTAL,'SEA' AS SRC FROM {TableList.FT_E_DTL_SEA_LIST.value} WHERE ISDELETED = 0
        AND HEAD_ID IN
        (SELECT ID FROM {TableList.FT_E_DTL_SEA_PRE_RECORDED.value} WHERE ISDELETED = 0 AND TRADE_CODE_SCC LIKE '{self.org_code}' AND I_E_DATE >= DATE'{self.startdt2}' AND I_E_DATE <= DATE'{self.enddt}')
        """
        CUS_SEA_LIST_E = Read_Oracle().read_oracle(sql=sql, database='dbdw')

        sql = f"""\
        SELECT ID,I_E_MARK,CUSTOMS_CODE,I_E_PORT,I_E_DATE,FIRST_DECL_TIME,TRAF_MODE,GOODS_OWNER FROM {TableList.FT_E_DTL_OTR_PRE_RECORDED.value} WHERE ISDELETED = 0 AND TRADE_CODE_SCC LIKE '{self.org_code}' AND I_E_DATE >= DATE'{self.startdt2}' AND I_E_DATE <= DATE'{self.enddt}'
        """
        CUS_OTR_HEAD_E = Read_Oracle().read_oracle(sql=sql, database='dbdw')

        sql = f"""\
        SELECT HEAD_ID, ENTRY_NO, GOODS_NO, GOODS_CODE, GOODS_NAME, GOODS_MODEL, 
        ORG_CODE_CY, G_QTY, G_UNIT, QTY_1, UNIT_1, QTY_2, UNIT_2, TRADE_CURR_CODE, 
        DECL_TOTAL,'OTR' AS SRC FROM {TableList.FT_E_DTL_OTR_LIST.value} WHERE ISDELETED = 0
        AND HEAD_ID IN
        (SELECT ID FROM {TableList.FT_E_DTL_OTR_PRE_RECORDED.value} WHERE ISDELETED = 0 AND TRADE_CODE_SCC LIKE '{self.org_code}' AND I_E_DATE >= DATE'{self.startdt2}' AND I_E_DATE <= DATE'{self.enddt}')
        """
        CUS_OTR_LIST_E = Read_Oracle().read_oracle(sql=sql, database='dbdw')

        sql = f"""\
        SELECT CY_CODE_ALPHY3,CY_CODE_CUS_NUMBER FROM {TableList.DIM_COUNTRY.value} WHERE ISCURRENT = 1 AND REGEXP_LIKE(CY_CODE_CUS_NUMBER,'^[0-9]*$') AND CY_CODE_CUS_NUMBER NOT LIKE '299' 
        """
        DIM_COUNTRY = Read_Oracle().read_oracle(sql=sql, database='dbdw')

        sql = f"""\
        SELECT CUS_CURRENCY_CODE,CURRENCY_CODE FROM {TableList.MAP_GJ_CURRENCY.value}
        """
        DIM_CURR = Read_Oracle().read_oracle(sql=sql, database='dbdw')

        CUS_SEA_IMPORT = pd.merge(CUS_SEA_HEAD_I,CUS_SEA_LIST_I,how='inner',left_on=['ID'],right_on=['HEAD_ID'])
        CUS_SEA_IMPORT = CUS_SEA_IMPORT.drop_duplicates(subset=['ENTRY_NO','GOODS_NO'])

        CUS_OTR_IMPORT = pd.merge(CUS_OTR_HEAD_I,CUS_OTR_LIST_I,how='inner',left_on=['ID'],right_on=['HEAD_ID'])
        CUS_OTR_IMPORT = CUS_OTR_IMPORT.drop_duplicates(subset=['ENTRY_NO','GOODS_NO'])

        CUS_SEA_EXPORT = pd.merge(CUS_SEA_HEAD_E,CUS_SEA_LIST_E,how='inner',left_on=['ID'],right_on=['HEAD_ID'])
        CUS_SEA_EXPORT = CUS_SEA_EXPORT.drop_duplicates(subset=['ENTRY_NO','GOODS_NO'])

        CUS_OTR_EXPORT = pd.merge(CUS_OTR_HEAD_E,CUS_OTR_LIST_E,how='inner',left_on=['ID'],right_on=['HEAD_ID'])
        CUS_OTR_EXPORT = CUS_OTR_EXPORT.drop_duplicates(subset=['ENTRY_NO','GOODS_NO'])

        CUS_DATA = pd.concat([CUS_SEA_IMPORT,CUS_OTR_IMPORT,CUS_SEA_EXPORT,CUS_OTR_EXPORT])
        CUS_DATA['TRADE_CURR_CODE'].replace(to_replace = list(DIM_CURR['CURRENCY_CODE']),value = list(DIM_CURR['CUS_CURRENCY_CODE']),inplace=True)
        CUS_SEA_IMPORT['TRADE_CURR_CODE'].replace(to_replace = list(DIM_CURR['CURRENCY_CODE']),value = list(DIM_CURR['CUS_CURRENCY_CODE']),inplace=True)

        CUS_DATA['ORG_CODE_CY'].replace(to_replace = list(DIM_COUNTRY['CY_CODE_ALPHY3']),value = list(DIM_COUNTRY['CY_CODE_CUS_NUMBER']),inplace=True)
        CUS_DATA = CUS_DATA.drop(columns=['ID','HEAD_ID'])
        CUS_DATA = CUS_DATA.reset_index().rename(columns = {'index':'ID'})
        now = datetime.datetime.strftime(datetime.datetime.now(), format='%Y-%m-%d %H:%M:%S')
        CUS_DATA['CHECK_TIME']=now
        CUS_DATA['CHECK_TIME']= pd.to_datetime(CUS_DATA['CHECK_TIME'], format='%Y-%m-%d %H:%M:%S')
        CUS_DATA['ORG_CODE'] = self.org_code

        STOCK_BILL['TRADE_TOTAL'] = STOCK_BILL['TRADE_TOTAL'].astype(float)
        STOCK_BILL['QTY'] = STOCK_BILL['QTY'].astype(float)
        STOCK_BILL['QTY_1'] = STOCK_BILL['QTY_1'].astype(float)
        STOCK_BILL['QTY_2'] = STOCK_BILL['QTY_2'].astype(float)

        stock_bill_sum = STOCK_BILL.groupby(['ORG_CODE','ENTRY_NO','ENTRY_GDS_SEQNO'])['QTY','QTY_1','QTY_2','TRADE_TOTAL'].sum()
        stock_bill_sum = stock_bill_sum.reset_index()
        stock_info = STOCK_BILL[['ORG_CODE','ENTRY_NO','ENTRY_GDS_SEQNO',
                                        'STOCK_BILL_TYPE','STOCK_TYPE','BILL_TYPE',
                                        'BUSINESS_TYPE','BILL_STATUS','CLASSIFY_TYPE',
                                        'SUPV_MODE','TRAF_MODE','I_E_PORT','CUSTOMS_CODE',
                                        'G_NAME','G_MODEL','ORIGIN_COUNTRY_CODE','HS_CODE','G_UNIT','UNIT_1',
                                        'UNIT_2','TRADE_CURR','STARTDT']].copy()

        stock_info = stock_info.drop_duplicates(subset=['ORG_CODE','ENTRY_NO','ENTRY_GDS_SEQNO'])
        stock_bill_clean = pd.merge(stock_bill_sum,stock_info,how='left',left_on=['ORG_CODE','ENTRY_NO','ENTRY_GDS_SEQNO'],
                        right_on=['ORG_CODE','ENTRY_NO','ENTRY_GDS_SEQNO'])
        stock_bill_clean['TRADE_CURR'].replace(to_replace=list(DIM_CURR['CURRENCY_CODE']),value=list(DIM_CURR['CUS_CURRENCY_CODE']),inplace=True)
        stock_bill_clean['ENTRY_GDS_SEQNO'] = stock_bill_clean['ENTRY_GDS_SEQNO'].astype(int)

        stock_bill_clean = stock_bill_clean.reset_index().rename(columns = {'index':'ID'})
        now = datetime.datetime.strftime(datetime.datetime.now(), format='%Y-%m-%d %H:%M:%S')
        stock_bill_clean['CHECK_TIME']=now
        stock_bill_clean['CHECK_TIME']= pd.to_datetime(stock_bill_clean['CHECK_TIME'], format='%Y-%m-%d %H:%M:%S')
        stock_bill_clean['TRADE_TOTAL'] = stock_bill_clean['TRADE_TOTAL'].astype(float)
        stock_bill_clean['QTY'] = stock_bill_clean['QTY'].astype(float)
        stock_bill_clean['QTY_1'] = stock_bill_clean['QTY_1'].astype(float)
        stock_bill_clean['QTY_2'] = stock_bill_clean['QTY_2'].astype(float)

        # 写入
        Write_Oracle().write_oracle(f'{TableList.BD_RISK_DETAIL_WAREHOUSE_WH1_C.value}',CUS_DATA,org_code=self.org_code, alarm=None)
        Write_Oracle().write_oracle(f'{TableList.BD_RISK_DETAIL_WAREHOUSE_WH1_S.value}',stock_bill_clean,org_code=self.org_code, alarm=None)

    def run_clean_wh1(self):
        try:
            self.clean_wh1()
            exec_status = 1
        except:
            logger.exception('model execution error')
            exec_status = 0
        finally:
            Risk_logger(child_task_id=self.child_task_id, exec_status=exec_status).write_log()


if __name__ == '__main__':
#     child_task_id = sys.argv[1]
    # 读取传入的 child_task_id，仅适用于py调用,先备注掉
    child_task_id = '0001_0005'
    org_code, param_json, base_time = read_log_table(child_task_id)
    org_code = '91310115688774070B'
    WarehouseCleanWh1(org_code, params=param_json, base_time = base_time, child_task_id=child_task_id).run_clean_wh1()

2021-06-04 14:30:01.761 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 11.15s
2021-06-04 14:30:19.303 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 17.53s
2021-06-04 14:30:33.469 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 14.16s
2021-06-04 14:31:38.398 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 64.92s
2021-06-04 14:32:29.002 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 50.59s
2021-06-04 14:33:27.293 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 58.28s
2021-06-04 14:34:41.693 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 -